In [1]:
import numpy as np
import scipy
import pandas as pd
from scipy import spatial
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
import time
from lmafit import lmafit_mc_adp


In [2]:
INVALID = 0.0

In [3]:
ratings = np.load('1M_ratings_np.npy')
movie_file = "ml-1m/movies.dat"
m_names = ["bad_index", "Title", "Genre"]
movies = pd.read_csv(movie_file, nrows=1_000_000, header=None, names=m_names, sep="::", engine='python')
movie_to_index = dict((m,i) for i,m in zip(movies.index, movies["bad_index"]))
movie_to_genre = dict((i,g) for i,g in zip(movies.index, movies["Genre"]))

genre_to_int = {
    'Action' : 0,
    'Adventure' : 1,
    'Animation' : 2,
    'Children\'s' : 3,
    'Comedy' : 4,
    'Crime' : 5,
    'Documentary' : 6,
    'Drama' : 7,
    'Fantasy' : 8,
    'Film-Noir' : 9, 
    'Horror' : 10,
    'Musical' : 11,
    'Mystery' : 12,
    'Romance' : 13,
    'Sci-Fi' : 14,
    'Thriller' : 15,
    'War' : 16,
    'Western' : 17, 
}
def get_movie_genres(movie):
    return [genre_to_int[genre] for genre in movie_to_genre[movie].split('|')]

def get_genres_for_movie(movie, curr):
    inds = [genre_to_int[genre] for genre in movie_to_genre[movie].split('|')]
    for i in inds:
        curr[i] += 1
    return curr

def get_top_k_genres_for_user(mat, user, k):
    movies = mat[user]
    genre_prefs = np.zeros(len(genre_to_int))
    if movies[np.nonzero(movies)].size == 0:
        return []
    average_rating = np.mean(movies[np.nonzero(movies)])
    for i,movie in enumerate(movies):
        if movie > average_rating:
            genre_prefs = get_genres_for_movie(i, genre_prefs)
    return np.flip(np.argsort(genre_prefs))[:k]

In [4]:
def RMSE(x, y):
    assert(x.shape == y.shape)
    return np.linalg.norm(x - y) / np.sqrt(x.shape[0])

def similarity_matrix_cosine(X):
    '''
    :param X: Ratings matrix. X[i, j] represents the rating of user i for item j.    
    :return: n x n cosine similarity matrix
    '''
    dist = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(X, 'cosine'))
    return 1 - dist

def similarity_matrix_knn(X, k, dim=2):
    '''
    :param X: Ratings matrix. X[i, j] represents the rating of user i for item j.    
    :return: n x n KNN similarity matrix
    '''
    n = X.shape[0]
    if k > n:
        raise IndexError('Only {} points, cannot have {} neighbors'
                         .format(n, k))
    sim = np.zeros((n, n))
    dist = spatial.distance_matrix(X, X, dim)
    for i in range(n):
        # ind is the indices of the closest k points to point i
        ind = np.argpartition(dist[i], -k)[:k]
        sim[i, ind] = 1.0 / k
    return sim

In [5]:
def get_rating(user_id, movie_id, ratings, sim):
    n = ratings.shape[0]
    sim_values = sim[user_id, :]
    movie_ratings = ratings[:, movie_id]
    
    # Use only valid ratings
    valid_ind = movie_ratings != INVALID
    sim_values = sim_values[valid_ind]
    movie_ratings = movie_ratings[valid_ind]
    pred_rating = sim_values.dot(movie_ratings)
    total = sim_values.sum()
    return pred_rating / total if total > 0 else pred_rating

In [6]:
true_ratings = np.load('1M_ratings_np.npy')



In [7]:
def test_collaborative_filtering(ratings, p, sim_measure='cosine', k=5, dim=2):
    '''
    :param ratings: Ratings matrix, where ratings[i][j] represents 
    user i's rating for movie j.
    :param sim_measure: Similarity measure to use
    :param p: Fraction of data to use as test-set
    :param k: In case of kNN, value of k to use
    :param dim: In case of kNN, value of dim to use
    
    :return: RMSE error of predictions
    '''
    # Get test indices
    valid_ind = np.where(ratings != INVALID)
    N = valid_ind[0].shape[0]
    test_subset = np.random.choice(np.arange(N), int(p * N), replace=False)
    test_ind = valid_ind[0][test_subset], valid_ind[1][test_subset]
    num_test = test_subset.shape[0]
    
    # Make the test indices invalid
    train_ratings = np.array(ratings)
    train_ratings[test_ind] = INVALID
    
    # Train on train set
    if sim_measure.lower() == 'cosine':
        sim = similarity_matrix_cosine(train_ratings)
    elif sim_measure.lower() == 'knn':
        sim = similarity_matrix_knn(train_ratings, k=k, dim=dim)
    else:
        raise ValueError('Unknown similarity measure {}'
                         .format(sim_measure))
    pred_ratings = np.zeros((num_test,))
    
    # Compute error
    true_ratings = ratings[test_ind]
    for i, (u, m) in enumerate(zip(*test_ind)):
        pred_ratings[i] = get_rating(u, m, train_ratings, sim)
    
    pred_ratings = pred_ratings.clip(1, 5).round()
    return RMSE(true_ratings, pred_ratings)

def test_matrix_comp(ratings, p):
    '''
    :param ratings: Ratings matrix, where ratings[i][j] represents 
    user i's rating for movie j.
    :param p: Fraction of data to use as test-set

    
    :return: RMSE error of predictions
    '''
    # Get test indices
    valid_ind = np.where(ratings != INVALID)
    N = valid_ind[0].shape[0]
    test_subset = np.random.choice(np.arange(N), int(p * N), replace=False)
    test_ind = valid_ind[0][test_subset], valid_ind[1][test_subset]
    num_test = test_subset.shape[0]
    
    # Make the test indices invalid
    train_ratings = np.array(ratings)
    train_ratings[test_ind] = INVALID
    
    # Train on train set
    tol = 1e-10
    thresh = 1e-2

    X = train_ratings[:,:]
    X_hat = np.zeros(X.shape)
    X_hat[X.nonzero()] = X[X.nonzero()]

    for k in range(25):
        X_old = np.copy(X_hat)
        u,s,v_t = np.linalg.svd(X_hat, full_matrices=False)
        s_hat = [i if i > thresh else 0 for i in s]
        X_hat = u @ np.diag(s_hat) @ v_t
        X_hat[X.nonzero()] = X[X.nonzero()]    
        if np.linalg.norm(X_hat - X_old) < tol:
            break
    pred_ratings = X_hat[test_ind]
    pred_new =  pred_ratings + np.abs(min(pred_ratings))
    pred_new *= 3.75e13    
    completed = pred_new.clip(1, 5).round()

    
    # Compute error
    true_ratings = ratings[test_ind]

    return RMSE(true_ratings, completed)
                

def test_matrix_completion(ratings, p, k):
    '''
    :param ratings: Ratings matrix, where ratings[i][j] represents 
    user i's rating for movie j.
    :param p: Fraction of data to use as test-set
    :param k: Estimated rank of matrix to use for LMaFit
    
    :return: RMSE error of predictions
    '''
    # Get test indices
    valid_ind = np.where(ratings != INVALID)
    N = valid_ind[0].shape[0]
    indices = np.arange(N)
    np.random.shuffle(indices)
    num_test = int(p * N)
    test_ind = valid_ind[0][indices[:num_test]], valid_ind[1][indices[:num_test]]
    train_ind = valid_ind[0][indices[num_test:]], valid_ind[1][indices[num_test:]]
        
    # Make the test indices invalid
    train_ratings = np.array(ratings)
    train_ratings[test_ind] = INVALID
    
    # Run LMaFit
    a, b, _ = lmafit_mc_adp(ratings.shape[0], ratings.shape[1], k, train_ind, ratings[train_ind], None)
    completed = a.dot(b)
    
    # Compute error
    completed = completed.clip(1, 5).round()
    return RMSE(ratings[test_ind], completed[test_ind])

def test_genre_matrix_completion(ratings, p, k, bias):
    '''
    :param ratings: Ratings matrix, where ratings[i][j] represents 
    user i's rating for movie j.
    :param p: Fraction of data to use as test-set
    :param k: Estimated rank of matrix to use for LMaFit
    
    :return: RMSE error of predictions with genre values
    '''
    # Get test indices
    valid_ind = np.where(ratings != INVALID)
    N = valid_ind[0].shape[0]
    indices = np.arange(N)
    np.random.shuffle(indices)
    num_test = int(p * N)
    test_ind = valid_ind[0][indices[:num_test]], valid_ind[1][indices[:num_test]]
    train_ind = valid_ind[0][indices[num_test:]], valid_ind[1][indices[num_test:]]
    
    # Make the test indices invalid
    train_ratings = np.array(ratings)
    train_ratings[test_ind] = INVALID
    # Run LMaFit
    a, b, _ = lmafit_mc_adp(ratings.shape[0], ratings.shape[1], k, train_ind, ratings[train_ind], None)
    completed = a.dot(b)
    topk = 3
    # Compute error
    for sample in np.array(test_ind).T:
        bias = bias
        user_genres = get_top_k_genres_for_user(train_ratings, sample[0], topk)
        movie_genres = np.array(get_movie_genres(sample[1]))
        sim_rating = len(np.intersect1d(user_genres, movie_genres))
        if (sim_rating > 0):
            completed[sample[0], sample[1]] += bias
        else:
            completed[sample[0], sample[1]] -= bias
    completed = completed.clip(1, 5).round()

    return RMSE(ratings[test_ind], completed[test_ind])



def test_avg_cross(ratings, p):
    '''
    :param ratings: Ratings matrix, where ratings[i][j] represents 
    user i's rating for movie j.
    :param p: Fraction of data to use as test-set

    :return: RMSE error of predictions averaged over movie scaled by user
    '''
    # Get test indices
    valid_ind = np.where(ratings != INVALID)
    N = valid_ind[0].shape[0]
    test_subset = np.random.choice(np.arange(N), int(p * N), replace=False)
    test_ind = valid_ind[0][test_subset], valid_ind[1][test_subset]
    num_test = test_subset.shape[0]
    
    # Make the test indices invalid
    train_ratings = np.array(ratings)
    train_ratings[test_ind] = INVALID
    
    # Train on train set
    pred_ratings = np.zeros((num_test,))
    overall_mean = train_ratings[np.nonzero(ratings)].mean()

    # Compute error
    true_ratings = ratings[test_ind]
    for i, (u, m) in enumerate(zip(*test_ind)):
        if train_ratings[u,:][np.nonzero(train_ratings[u,:])].shape[0] > 0:       
            user_scaled_avg = train_ratings[u,:][np.nonzero(train_ratings[u,:])].mean() / overall_mean
            if train_ratings[:,m][np.nonzero(train_ratings[:,m])].shape[0] > 0:
                pred_ratings[i] = train_ratings[:,m][np.nonzero(train_ratings[:,m])].mean()
            else:
                pred_ratings[i] = user_scaled_avg
        else:
            pred_ratings[i] = overall_mean
    
    pred_ratings = pred_ratings.clip(1, 5).round()
    return RMSE(true_ratings, pred_ratings)


In [9]:
biases = [0.0, 0.1, 0.2, 0.5, 1.0]
p_vals = [0.95, 0.99]
k = 2
slce = None

results = []
for p in p_vals:
    curr = []
    print("------------- p=" + str(p) + "--------------")
    lma = test_matrix_completion(ratings=np.copy(true_ratings[:slce, :slce]), p=p, k=k)
    curr.append(lma)
    print("LMaFit " + " " + str(lma))
    for b in biases:
        bs = test_genre_matrix_completion(ratings=np.copy(true_ratings[:slce, :slce]), p=p, k=k, bias=b)
        curr.append(bs)
        print("Genre "+ str(b) + " " + str(bs))
    results.append(curr)
print(results)

------------- p=0.95--------------
LMaFit  1.28548413980267
Genre 0.0 1.2637833758412205
Genre 0.1 1.2992767218725965
Genre 0.2 1.302794163165827
Genre 0.5 1.3271619898747933
Genre 1.0 1.5103011202022631
------------- p=0.99--------------
LMaFit  2.1145343590387826
Genre 0.0 2.116923868887747
Genre 0.1 2.126947354897401
Genre 0.2 2.12229269014108
Genre 0.5 2.1385504389183763
Genre 1.0 2.1886458615022
[[1.28548413980267, 1.2637833758412205, 1.2992767218725965, 1.302794163165827, 1.3271619898747933, 1.5103011202022631], [2.1145343590387826, 2.116923868887747, 2.126947354897401, 2.12229269014108, 2.1385504389183763, 2.1886458615022]]


In [8]:
slce = None
ranks = [1, 2, 3, 4, 8, 16, 32, 64, 128]
# p_vals = [0.2, 0.4, 0.6, 0.8]
p_vals = [0.9, 0.95, 0.99]

results2 = []
for p in p_vals:
    curr = []
    print("------------- p=" + str(p) + "--------------")
    isvt = test_matrix_comp(ratings=np.copy(true_ratings[:slce, :slce]), p=p)
    curr.append(isvt)
    print("ISVT "  + " " + str(isvt))
    plus_avg = test_avg_cross(ratings=np.copy(true_ratings[:slce, :slce]), p=p)
    curr.append(plus_avg)
    print("+_avg " + " " + str(plus_avg))
    for r in ranks:
        lma = test_matrix_completion(ratings=np.copy(true_ratings[:slce, :slce]), p=p, k=r)
        curr.append(lma)
        print("LMaFit rank:"+ str(r) + " " + str(lma))
    results2.append(curr)
print(results2)

------------- p=0.9--------------
ISVT  1.2632053585137206
+_avg  1.0485413148221145
LMaFit rank:1 1.0407080922782013
LMaFit rank:2 1.0776852766719769
LMaFit rank:3 1.120553930478632
LMaFit rank:4 1.1818572953900428
LMaFit rank:8 1.3690312876872708
LMaFit rank:16 1.7533019642314263
LMaFit rank:32 2.1802589040957705
LMaFit rank:64 2.4589220763939994
LMaFit rank:128 2.6544681409109265
------------- p=0.95--------------
ISVT  2.8127424339956906
+_avg  1.110595857139192
LMaFit rank:1 1.1172701694661624
LMaFit rank:2 1.2889771546707713
LMaFit rank:3 1.4147594178963137
LMaFit rank:4 1.4971748834317744
LMaFit rank:8 1.8579383027544552
LMaFit rank:16 2.249553639935569
LMaFit rank:32 2.4983474538182238
LMaFit rank:64 2.661421859794023
LMaFit rank:128 2.7561233262453415
------------- p=0.99--------------
ISVT  2.8127123039960034
+_avg  1.6123832664660713
LMaFit rank:1 1.7247412337021235
LMaFit rank:2 2.116984943858896
LMaFit rank:3 2.29902021291548
LMaFit rank:4 2.394273682068628
LMaFit rank:8 2

In [9]:
print(results)

[[0.9352031864787459, 0.9414695958978176, 0.9631355044852205, 0.9585379491705063, 1.048174603775535, 1.3153592665123852], [0.9452658356250901, 0.9462425693235322, 0.9518757797107771, 0.9769953940526024, 1.0630475059939701, 1.3078598931078205], [0.9795219922662959, 0.9659537601079395, 0.9708115162069308, 1.0856096597457732, 1.074263313469592, 1.3151324394650652]]
